### Exercice 1 – String Edit Distance
On vous demande dans cet exercice de programmer la version recursive du calcul de distance de Levenshtein par programmation dynamique. Les plus téméraires pourront s’attaquer à la version Bottom-up.

In [1]:
def EditDistCost(s1, s2, i, j):
    m = len(s1)
    n = len(s2)

    # Final Objective
    if i == m and j == n:
        return 0
    
    # If the first letter is finished
    elif i == m:
        return abs(n) - j
    
    # If the second letter is finished
    elif j == n:
        return abs(m) - i
    
    # If letters are the same, pass to the following
    elif s1[i] == s2[j]:        
        return EditDistCost(s1, s2, i+1, j+1)

    # If letters are different
    else: 
        mincost = min(
                    EditDistCost(s1, s2, i+1, j) + 1 #del
                    , min(
                        EditDistCost(s1, s2, i, j+1) + 1 #ins
                        , EditDistCost(s1, s2, i+1, j+1) + 1 #sub                      
                        ) 
                    )

        return mincost

In [2]:
s1 = "intention"
s2 = "execution"

m = len(s1)
n = len(s2)

cost = EditDistCost(s1, s2, 0, 0)

print("Levenshtein Distance:", cost)

Levenshtein Distance: 5


### Exercice 2 – 0/1 Knapsack problem
On vous demande dans cet exercice de programmer la version récursive de résolution du problème du sac à dos 0/1 par programmation dynamique.

Vous pourrez tester votre algorithme sur les données fournies sur http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/ qui nous serviront de benchmark pour les TP suivants.

In [3]:
import numpy as np
import time
import os

In [4]:
def Knapsack_NoMemo(itens, i, X):

    if i > len(itens)-1:
        return 0

    wi = itens[i][1]
    ri = itens[i][0]

    p1 = Knapsack_NoMemo(itens,i+1, X)

    if wi <= X:
        p2 = ri + Knapsack_NoMemo(itens,i+1, X-wi)
    else: 
        p2 = 0

    return max(p1, p2)


In [5]:
def Knapsack(itens, i, X, memo):
    if i > len(itens) - 1:
        return 0
    
    if (i, X) in memo:
        return memo[(i, X)]

    wi = itens[i][1]
    ri = itens[i][0]

    p1 = Knapsack(itens, i + 1, X, memo)
    
    if wi <= X:
        p2 = ri + Knapsack(itens, i + 1, X - wi, memo)
    else:
        p2 = 0
    
    memo[(i, X)] = max(p1, p2)
    return memo[(i, X)]

In [6]:
datasets = []
listing = os.listdir("./instances_01_KP/low-dimensional/")
my_array=np.zeros(shape=(0,5))
for infile in listing:
    path = "./instances_01_KP/low-dimensional/" + infile
    data = np.loadtxt(path)
    datasets.append(data)

In [7]:
for dataset in datasets:
    n = dataset[0][0]
    wmax = dataset[0][1]
    itens = dataset[1:]

    memo = {}

    print(f"\n===Problem with {n:.0f} itens and {wmax:.0f} capacity===")
    tic = time.time()
    rmax = Knapsack(itens, 0, wmax, memo)
    toc = time.time()

    # tic2 = time.time()
    # rmax2 = Knapsack_NoMemo(itens, 0, wmax)
    # toc2 = time.time()

    print("Optimum value = ", rmax)

    print(f"With Memoizaion, compute in {(toc - tic):.4f} seconds")
    # print(f"Without Memoizaion, compute in {(toc2 - tic2):.4f} seconds")


===Problem with 15 itens and 375 capacity===
Optimum value =  481.069368
With Memoizaion, compute in 0.0370 seconds

===Problem with 10 itens and 269 capacity===
Optimum value =  295.0
With Memoizaion, compute in 0.0004 seconds

===Problem with 10 itens and 60 capacity===
Optimum value =  52.0
With Memoizaion, compute in 0.0002 seconds

===Problem with 20 itens and 878 capacity===
Optimum value =  1024.0
With Memoizaion, compute in 0.0064 seconds

===Problem with 4 itens and 11 capacity===
Optimum value =  23.0
With Memoizaion, compute in 0.0000 seconds

===Problem with 23 itens and 10000 capacity===
Optimum value =  9767.0
With Memoizaion, compute in 0.0097 seconds

===Problem with 5 itens and 80 capacity===
Optimum value =  130.0
With Memoizaion, compute in 0.0000 seconds

===Problem with 7 itens and 50 capacity===
Optimum value =  107.0
With Memoizaion, compute in 0.0001 seconds

===Problem with 20 itens and 879 capacity===
Optimum value =  1025.0
With Memoizaion, compute in 0.0062

In [18]:
datasets = []
N_values = []
W_values = []
listing = os.listdir("./instances_01_KP/large_scale/")
my_array=np.zeros(shape=(0,5))
for infile in listing:
    path = os.path.join("./instances_01_KP/large_scale/", infile)
    
    parts = infile.split("_")
    N = int(parts[2])  # Terceiro elemento corresponde a N
    W = int(parts[3])  # Quarto elemento corresponde a W
    
    N_values.append(N)
    W_values.append(W)

    with open(path, "r") as f:
        lines = f.readlines()[:-1]  
    
    data = np.loadtxt(lines)
    datasets.append(data)

In [19]:
print(N_values)

[200, 2000, 500, 100, 1000]


In [20]:
i=0
for dataset in datasets:
    n = N_values[i]
    wmax = W_values[i]
    itens = dataset[1:]

    memo = {}

    print(f"\n===Problem with {n:.0f} itens and {wmax:.0f} capacity===")
    tic = time.time()
    rmax = Knapsack(itens, 0, wmax, memo)
    toc = time.time()

    # tic2 = time.time()
    # rmax2 = Knapsack_NoMemo(itens, 0, wmax)
    # toc2 = time.time()

    print("Optimum value = ", rmax)

    print(f"With Memoizaion, compute in {(toc - tic):.4f} seconds")
    # print(f"Without Memoizaion, compute in {(toc2 - tic2):.4f} seconds")
    i = i+1


===Problem with 200 itens and 1000 capacity===
Optimum value =  11238.0
With Memoizaion, compute in 0.1286 seconds

===Problem with 2000 itens and 1000 capacity===
Optimum value =  33449.0
With Memoizaion, compute in 1.9746 seconds

===Problem with 500 itens and 1000 capacity===
Optimum value =  18582.0
With Memoizaion, compute in 0.4072 seconds

===Problem with 100 itens and 1000 capacity===
Optimum value =  9147.0
With Memoizaion, compute in 0.0499 seconds

===Problem with 1000 itens and 1000 capacity===
Optimum value =  24091.0
With Memoizaion, compute in 0.9268 seconds
